Introduction:
1. Data Preprrocessing techniques:
    a. image resize
    b. Normalization 
    c. Data Augmentation (Example: Horizontal Flipping, rotation, scaling )
    d. Histogram Equalization
    e. Data Cleaning (Example: Removing images with low contrast)
    f. Noise Reduction (bluring and denoising)
    g. Splitting the Dataset (Example: 80-10-10 split)


Lets split the data first into training, validation and testing using the 75%, 15%, 15% respectively.

In [3]:
import os
import shutil
import random

# Get the directory of the current script
script_dir = os.path.dirname(os.path.abspath(__file__))

# Define paths relative to the script location
data_dirOuter = os.path.join(script_dir, 'data')
data_dir = os.path.join(data_dirOuter, 'data')  # Adjust this path to point to the nested data folder
train_dir = os.path.join(data_dirOuter, 'train')  # Train directory inside the first data folder
val_dir = os.path.join(data_dirOuter, 'val')      # Validation directory inside the first data folder
test_dir = os.path.join(data_dirOuter, 'test')

# Define the split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Create directories for train, val, and test sets
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Loop through each class folder
for class_name in os.listdir(data_dir):
    class_path = os.path.join(data_dir, class_name)
    if os.path.isdir(class_path):
        images = os.listdir(class_path)
        random.shuffle(images)
        
        # Calculate split indices
        train_split = int(train_ratio * len(images))
        val_split = int((train_ratio + val_ratio) * len(images))
        
        # Move images to respective directories
        for i, image in enumerate(images):
            src_path = os.path.join(class_path, image)
            if i < train_split:
                dst_dir = os.path.join(train_dir, class_name)
            elif i < val_split:
                dst_dir = os.path.join(val_dir, class_name)
            else:
                dst_dir = os.path.join(test_dir, class_name)
            
            os.makedirs(dst_dir, exist_ok=True)
            shutil.copy(src_path, dst_dir)

